In [30]:
import http.client

conn = http.client.HTTPSConnection("api.apihighways.org")

conn.request("GET", "/query?sql=SELECT%20*%20FROM%20aa1c5c53-cb73-4b29-94f4-5b1695d37dc5")

res = conn.getresponse()
data = res.read()

print(data.decode("utf-8"))

{"data":[{"1960":0,"1961":0,"1962":0,"1963":0,"1964":0,"1965":0,"1966":0,"1967":0,"1968":0,"1969":0,"1970":0,"1971":0,"1972":0,"1973":0,"1974":0,"1975":0,"1976":0,"1977":0,"1978":0,"1979":0,"1980":0,"1981":0,"1982":0,"1983":0,"1984":0,"1985":0,"1986":0,"1987":0,"1988":0,"1989":0,"1990":0,"1991":0,"1992":0,"1993":0,"1994":0,"1995":0,"1996":0,"1997":0,"1998":0,"1999":0,"2000":0,"2001":0,"2002":0,"2003":0,"2004":0,"2005":0,"2006":0,"2007":0,"2008":0,"2009":0,"2010":0,"2011":0,"2012":0,"2013":0,"2014":0,"2015":0,"2016":0,"Country_Name":"Andorra","Country_Code":"AND","Indicator_Name":"Poverty headcount ratio at $1.90 a day (2011 PPP) (% of population)","Indicator_Code":"SI.POV.DDAY","_id":"AVmXp2sRGMS00IEWB7HJ"},{"1960":0,"1961":0,"1962":0,"1963":0,"1964":0,"1965":0,"1966":0,"1967":0,"1968":0,"1969":0,"1970":0,"1971":0,"1972":0,"1973":0,"1974":0,"1975":0,"1976":0,"1977":0,"1978":0,"1979":0,"1980":0,"1981":0,"1982":0,"1983":0,"1984":0,"1985":0,"1986":0,"1987":0,"1988":0,"1989":0,"1990":0,"19

In [31]:
import json
j = json.loads(data)
j.keys()

dict_keys(['data', 'meta'])

In [32]:
import pandas as pd
#df = pd.read_json(data["data"], orient='records')     # "TypeError: byte indices must be integers or slices, not str"

df = pd.DataFrame(j["data"])
df.head()

,1960,1961,1962,1963,1964,1965,1966,1967,1968,1969,...,2012,2013,2014,2015,2016,Country_Code,Country_Name,Indicator_Code,Indicator_Name,_id
0,0,0,0,0,0,0,0,0,0,0,...,0.00,0.0,0.0,0,0,AND,Andorra,SI.POV.DDAY,Poverty headcount ratio at $1.90 a day (2011 P...,AVmXp2sRGMS00IEWB7HJ
1,0,0,0,0,0,0,0,0,0,0,...,0.00,0.0,0.0,0,0,AFG,Afghanistan,SI.POV.DDAY,Poverty headcount ratio at $1.90 a day (2011 P...,AVmXp2sRGMS00IEWB7HK
2,0,0,0,0,0,0,0,0,0,0,...,0.00,0.0,0.0,0,0,AGO,Angola,SI.POV.DDAY,Poverty headcount ratio at $1.90 a day (2011 P...,AVmXp2sRGMS00IEWB7HL
3,0,0,0,0,0,0,0,0,0,0,...,1.06,0.0,0.0,0,0,ALB,Albania,SI.POV.DDAY,Poverty headcount ratio at $1.90 a day (2011 P...,AVmXp2sRGMS00IEWB7HM
4,0,0,0,0,0,0,0,0,0,0,...,0.00,0.0,0.0,0,0,ARE,United Arab Emirates,SI.POV.DDAY,Poverty headcount ratio at $1.90 a day (2011 P...,AVmXp2sRGMS00IEWB7HO


In [33]:
# "enables the %%R magic, not necessary if you've already done this"
%load_ext rpy2.ipython

The rpy2.ipython extension is already loaded. To reload it, use:
  %reload_ext rpy2.ipython


In [34]:
%%R
library(dplyr)
head(df, 5)

#library(ggplot2)
#ggplot(df, aes(x=Country_Name, y=Country_Code)) +
#  geom_point()

# Gives an error. I was trying to follow [https://stackoverflow.com/questions/39008069/r-and-python-in-one-jupyter-notebook]

                                              
1 function (x, df1, df2, ncp, log = FALSE)    
2 {                                           
3     if (missing(ncp))                       
4         .Call(C_df, x, df1, df2, log)       
5     else .Call(C_dnf, x, df1, df2, ncp, log)
